Let's solve the scalar diffusion equation on $(t,x)\in[0,\infty)\times[-1,1]$,

$$\partial_t q(t,x) - \nu \partial_{xx} q(t,x) = 0.$$

Starting with a constant $\nu$, using a Chebyshev polynomial basis of the first kind ($T_n(x)$) and a test basis of Ultraspherical polynomials of the second kind ($C^2_n(x)$).  Thus, the diffusion equation becomes

$$\partial_t \sum_{n=0}^{\infty} q_n(t) T_n(x) - \nu \partial_{xx}\sum_{n=0}^{\infty} q_n(t) T_n(x) = 0.$$

Since we are solving problems on a finite computer, the spatial series representation of the scalar must be truncated to N terms, permitting the commutation of the sums and derivatives. Then,

$$\sum_{n=0}^{N} q_n'(t) T_n(x) - \frac{\nu}{2} \sum_{n=2}^{N} n q_n(t) C^2_n(x)  = 0.$$

So, to convert the Chebyshev polynomial $T_n(x)$ to the Ultraspherical polynomial we need two matrices: one to convert the $T_n(x)$ to the $C^1_n(x)$, which are equivalent to the Chebyshev polynomials of the second kind, and then the second conversion from $C^1_n(x)$ to $C^2_n(x)$.  The first matrix $S_0$, follows from Chebyshev recurrence relationships and is

$$
S^0 = \begin{bmatrix} 
1 & 0 & -1/2 & 0 &\cdots \\
0 & 1/2 & 0 & -1/2 & 0 &\cdots \\
0 & 0 & \ddots & \ddots & \ddots
\end{bmatrix}
$$

The second connection between derivative orders $\lambda$ as $C_n^\lambda(x)$ and $C_n^{\lambda+1}(x)$ can be found in Olver & Townsend 2013, with

$$
S^\lambda = \begin{bmatrix} 
1 & 0 & -\frac{\lambda}{\lambda+2} & 0 &\cdots \\
0 & \frac{\lambda}{\lambda+1} & 0 & -\frac{\lambda}{\lambda+3} & 0 &\cdots \\
0 & 0 & \frac{\lambda}{\lambda+2} & 0 & -\frac{\lambda}{\lambda+4} & 0 &\cdots \\
0 & 0 & \ddots & \ddots & \ddots
\end{bmatrix}
$$

Thus, we can write the equation as

$$\sum_{n=0}^{N}\sum_{i}\left[\sum_{j}S^1_{ni} S^0_{ij} q_j'(t)-\nu D^2_{ni}q_i(t)\right] C^2_n(x)  = 0,$$

where the $\lambda$-order derivative matrix $D^\lambda$ has the first $\lambda$ columns as zeros and then the single super-diagonal pattern 

$$
D^\lambda =\begin{bmatrix}
0&\cdots & 0& \lambda & 0 & \cdots \\
0&\cdots & 0& 0& \lambda+1 & 0 & \cdots \\
0&\cdots & 0& 0& 0& \lambda+2 & 0 & \cdots\\
\vdots& & \cdots & & & &\ddots & 0 & \cdots
\end{bmatrix}
$$

The next step is to approximate the time derivative.  Let's use Crank-Nicholson:


$$\sum_{n=0}^{N}\sum_{i}\left[\sum_{j}S^1_{ni} S^0_{ij} \frac{(q_j^{k+1}-q_j^k)}{\Delta t}-\frac{\nu}{2}D^2_{ni}(q_i^{k+1}+q_i^k)\right] C^2_n(x)  = 0.$$

Or in matrix form for the coefficients (e.g. integrating against the $C^2_n(x)$), we have

$$
\left[S^1 S^0 - \frac{\Delta t \nu}{2}D^2\right]\mathbf{q}^{k+1} = \left[S^1 S^0 + \frac{\Delta t \nu}{2}D^2\right]\mathbf{q}^k,\\
A_{-}\mathbf{q}^{k+1} = A_{+}\mathbf{q}^k.
$$

We then need to incorporate boundary conditions.  This can be done in several ways... via taus (extra degrees of freedom) or via row deletion.  Olver and Townsend use row deletion. So, we'll go with that for now (plus it is closer to the current Rayleigh implemention). The coefficient condition will be implemented on the left-hand side of the equation and the boundary values will replace some rows on the right-hand side. For instance, letting $B_2$ be the boundary operator, $P_{N-2}$ a restriction operator that removes the last two rows of $A_{-}$ and $A_{+}$, and $\mathbf{c}$ be the boundary values, the matrix equations become

$$
\begin{bmatrix}
B_2\\
P_{N-2}A_{-}
\end{bmatrix}\mathbf{q}^{k+1} = 
\begin{bmatrix}
\mathbf{0}\\
P_{N-2}A_{+}
\end{bmatrix}\mathbf{q}^k + \mathbf{c}.
$$

Therefore, after inverting the left-hand side matrix via LU factorization, the solution at $t_{k+1}$ is then

$$q(t_{k+1},x)=\sum_{n=0}^N q_n^{k+1}T_n(x) + \mathcal{O}\left(\Delta t^2, \frac{\partial^{N+1} q}{\partial x^{N+1}}\right).$$

If the diffusivity is allowed to vary in space, then we need to include the multiplication matrices, and slightly change our equation to account for Fick's diffusion law, e.g.

$$\partial_t q(t,x) - \nu(x) \partial_{xx} q(t,x) - \partial_x\nu(x) \partial_x q(t,x) = 0.$$

The multiplication matrices link two scalar functions $a(x)$ and $u(x)$ to the same Ultraspherical bases, with

$$a(x)u(x) = \sum_{j=0}^M\sum_{k=0}^N\sum_{s=\mathrm{max}(0,k-j)}^k a_{2 s + j - k} c_s^\lambda(k,2 s + j -k)u_k C^\lambda_s(x),$$

where the $c_s^\lambda$ are the connection coefficients (see Olver & Townsend 2013 for details).  That is the multiplication matrix $M^\lambda[a]$ has the elements

$$M^\lambda[a]_{j,k} = \sum_{s=\mathrm{max}(0,k-j)}^k a_{2 s + j - k} c_s^\lambda(k,2 s + j -k).$$

Note here $a(x)$ is expressed on the $C^\lambda(x)$ basis and so if one defines this scalar over the Chebyshev polynomials of the first kind, then one must create additional conversion matrices to bring this field onto the appropriate Ultraspherical basis.

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

In [ ]:
def dct_chebyshev_transform(f):
    N = len(f)
    coefs = f
    coefs = sp.fft.dct(coefs,orthogonalize=True,norm='ortho')/np.sqrt(0.5*N)
    coefs[0] = coefs[0]/np.sqrt(2)
    return coefs

def idct_chebyshev_transform(f):
    N = len(f)
    icoefs = np.sqrt(N/2)*f
    icoefs[0] = np.sqrt(2)*icoefs[0]
    icoefs = sp.fft.idct(icoefs,orthogonalize=True,norm='ortho')
    return icoefs

def get_collocation_points(x0,x1,N):
    #Map to Chebyshev domain
    x = np.pi*(np.arange(N)+0.5)/np.float64(N)
    x = 0.5*(x1+x0)-0.5*(x1-x0)*np.cos(x)
    return x

def multiplication_coef(s,j,k,lam):
    t1 = j + k + lam
    t2 = t1 - s
    t1 = t1 - 2*s
    
    #First term
    cs = t1/t2
    
    t2 = 1
    for t in range(s - 1):
        t2 = t2*(lam + t)/(1 + t)
    
    #Second term
    cs = cs*t2
    
    t3 = 1 
    for t in range(j - s - 1):
        t3 = t3*(lam + t)/(1 + t)
    
    #Third term
    cs = cs*t3
    
    t4 = 1
    for t in range(s - 1):
        t1 = j + k - 2*s + t
        t2 = t1 + lam
        t1 = t1 + 2*lam
        t1 = t1/t2
        t4 = t4*t1
    
    #Fourth term
    cs = cs*t4
    
    t5 = 1
    for t in range(j-s-1):
        t1 = k - s + t
        t2 = t1 + lam
        t1 = t1 + 1
        t1 = t1/t2
        t5 = t5*t1
        
    #Fifth term
    cs = cs*t5
    return cs

def build_multiplication_matrix(coefs,N,lam):
    Mlam = sparse.lil_matrix((N,N),dtype=np.float64)
    ncoef = len(coefs)
    for j in range(N):
        for k in range(N):
            t = 0
            i1 = int(np.max([0,k-j]))
            inds = np.array(2*np.arange(i1,k,1)+j-k).astype(int)
            ainds = np.where((inds>=0)&(inds<ncoef))[0]
            sinds = np.arange(i1,k,1,dtype=int)
            for a in ainds:
                cs = multiplication_coef(sinds[a],k,inds[a],lam)
                t = t + cs*coefs[inds[a]]
            Mlam[j,k] = t
    Mlam.tocsc()
    return Mlam

def build_derivative_matrix(N,lam):
    elements = 2**(lam-1)*np.math.factorial(lam-1)*(np.arange(N,dtype=np.float64)+lam)
    Dlam = sparse.diags(elements,lam,shape=(N,N),format='csc')
    return Dlam
    
def build_conversion_matrix(N,lam):
    if (lam==0):
        upper = -np.ones(N-2)*0.5
        diag = np.ones(N)*0.5
        diag[0] = 1
        Slam = sparse.diags([diag,upper],[0,2],format='csc')
    else:
        ran = np.arange(N-2)+lam+2
        upper = -lam*np.ones(N-2)/ran
        ran = np.arange(N)+lam
        diag = lam*np.ones(N)/ran
        diag[0] = 1
        Slam = sparse.diags([diag,upper],[0,2],format='csc')   
    return Slam

def build_boundary_matrix(N):
    #Dirichlet conditions
    #x=-1
    xm1 = np.zeros(N,dtype=int)
    xp1 = np.ones(N,dtype=int)
    for n in range(N):
        xm1[n] = 1 - 2*(n % 2)
    xm1.astype(np.float64)
    Blam = sparse.lil_matrix((N,N),dtype=np.float64)
    Blam[N-2,:] = xm1
    Blam[N-1,:] = xp1
    return Blam.tocsc()
    
def build_permutation_matrix(N):
    Plam = sparse.lil_matrix((N,N),dtype=np.float64)
    for ii in range(N-2):
        Plam[ii+2,ii] = 1
    Plam[0,N-1] = 1
    Plam[1,N-2] = 1
    return Plam.tocsc()

def build_preconditioner(N):
    ran = np.arange(N-2)+2
    elements = 0.5*np.ones(N,dtype=np.float64)
    elements[0:N-2] = 1/ran
    Pr = sparse.diags(elements,0,shape=(N,N),dtype=np.float64,format='csc')
    return Pr

#Time parameters
ntimes = 200
deltat = 1e-3
times = np.arange(ntimes)*deltat

#Diffusivity
nu = 1

#Truncation size
N = 128
Ndealias = int(N*3/2)

#Chebyshev collocation points (roots), excludes boundaries
xi = -np.cos(np.pi*(np.arange(N)+0.5)/N)

#Initial condition in physical space, and satisfying the boundary conditions.
q = np.sin(2*np.pi*xi)

#Test DCT
forward = dct_chebyshev_transform(q)
backward = idct_chebyshev_transform(forward)
#print('DCT error =',np.max(np.abs(q-backward)))

#Chebyshev transform
qcoefs = np.zeros(Ndealias,dtype=np.float64)
qcoefs[0:N] = dct_chebyshev_transform(q)

#Decompose the problem in spectral space
L = build_derivative_matrix(Ndealias,2)
S1 = build_conversion_matrix(Ndealias,1)
S0 = build_conversion_matrix(Ndealias,0)
B = build_boundary_matrix(Ndealias)
#P = build_permutation_matrix(Ndealias)
Pr = build_preconditioner(Ndealias)

#Mass matrix
M = S1@S0
#plt.spy(M)
#plt.show()

#Linear operator
ncc_bool = False
if (ncc_bool):
    #First order term
    #dnu(x)/dx = nu * df(x)/dx = nu*4*x
    D1 = build_derivative_matrix(Ndealias,1)
    ncc = 4*xi
    ncc_coefs = dct_chebyshev_transform(ncc)
    ncc_coefs[np.where(np.abs(ncc_coefs)<1e-10)] = 0
    nncc = np.max(np.where(np.abs(ncc_coefs))[0])+1
    ncc_coefs = ncc_coefs[0:nncc]
    S0m = build_conversion_matrix(nncc,2)
    S1m = build_conversion_matrix(nncc,2)
    #Convert to C^2(x) basis from T_n(x), truncated to have as few coefs as possible.
    C = S1m@S0m
    ncc_coefs = C.dot(ncc_coefs)
    NC1 = build_multiplication_matrix(ncc_coefs,Ndealias,1)
    
    #Second order term
    #nu(x) = nu * f(x) = nu * (2*x^2 + 1)
    ncc = 2*xi**2 + 1
    ncc_coefs = dct_chebyshev_transform(ncc)
    ncc_coefs[np.where(np.abs(ncc_coefs)<1e-10)] = 0
    nncc = np.max(np.where(np.abs(ncc_coefs))[0])+1
    ncc_coefs = ncc_coefs[0:nncc]
    S0m = build_conversion_matrix(nncc,2)
    S1m = build_conversion_matrix(nncc,2)
    #Convert to C^2(x) basis from T_n(x), truncated to have as few coefs as possible.
    C = S1m@S0m
    ncc_coefs = C.dot(ncc_coefs)
    NC2 = build_multiplication_matrix(ncc_coefs,Ndealias,2)
    
    L = 0.5*deltat*nu*(NC2@L + S1@NC1@D1)
else:
    L = 0.5*deltat*nu*L
#plt.spy(L)
#plt.show()

#Define the problem matrices
Am = M - L
Ap = M + L

#Omit the two highest order rows of A, which are replaced with the boundary operators
Am[Ndealias-2:Ndealias-1,:] = 0
Am = Am+B

#Precondition the matrices
Am = Pr@Am
Ap = Pr@Ap

#Row permutation to move the last two rows to the top of the matrix
#Am = P@Am
plt.spy(Am)
plt.show()

#LU Decompose Am
AmLU = sparse.linalg.splu(Am,permc_spec='COLAMD')

qk=qcoefs
numer = np.zeros((N,ntimes),dtype=np.float64)
exact = np.zeros((N,ntimes),dtype=np.float64)
n = 0

for t in times:
    #qk[N:] = 0
    exact[:,n] = q*np.exp(-4*np.pi**2*t)
    numer[:,n] = idct_chebyshev_transform(qk[0:N])
    qk = Ap.dot(qk)
    #Enforce boundary conditions
    qk[Ndealias-2:] = 0
    qk = AmLU.solve(qk)
    n = n+1

fig, ax = plt.subplots(1,2)
line, = ax[0].plot([],color='r')
line2, = ax[0].plot([],color='b')
line3, = ax[1].plot([])
time_template = 't = %.3f'
time_text = ax[0].text(0.05, 0.95, '', transform=ax[0].transAxes)
#err_template = r'$\log_{10}(e)$'+' = %.1f'
#err_text = ax[0].text(0.7, 0.95, '', transform=ax.transAxes)
ax[0].set_xlim(-1,1)
ax[0].set_ylim(-1,1)
ax[1].set_xlim(-1,1)
maxerr = np.max(np.abs(numer-exact))
ax[1].set_ylim((-maxerr,maxerr))

def animate(frame_num):
    y = numer[:,frame_num]
    line.set_data((xi,numer[:,frame_num]))
    line2.set_data((xi,exact[:,frame_num]))
    line3.set_data((xi,numer[:,frame_num]-exact[:,frame_num]))
    time_text.set_text(time_template % (times[frame_num]))
    #err_text.set_text(err_template % (np.log10(np.max(np.abs(numer[:,frame_num]-exact[:,frame_num])/np.abs(exact[:,frame_num])))))
    return line, line2, time_text

anim = FuncAnimation(fig,animate,frames=ntimes,interval=50)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()                   # avoid plotting a spare static plot